<a href="https://www.kaggle.com/code/tunguz/toxic-comments-with-embeddings-and-rapids-lr?scriptVersionId=108571756" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cuml, cudf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_320.npy
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from cuml.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from scipy.sparse import hstack
from sklearn.metrics import roc_auc_score

In [3]:
%%time
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

target = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip').fillna(' ')[class_names].values
target

CPU times: user 1.21 s, sys: 100 ms, total: 1.31 s
Wall time: 1.64 s


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [4]:
train_features = np.load('../input/toxic-comments-transformer-embeddings/train_embs_large_400.npy')
test_features = np.load('../input/toxic-comments-transformer-embeddings/test_embs_large_400.npy')

In [5]:
%%time
train_oof = np.zeros(target.shape)
test_preds = np.zeros((test_features.shape[0],6))
N_FOLDS = 10
kf = KFold(n_splits=N_FOLDS, random_state=137, shuffle=True)
for ii in range(6):
    print("Fitting target", ii+1)
    for jj, (train_index, val_index) in enumerate(kf.split(train_features)):
        print("Fitting fold", jj+1)
        train_x = train_features[train_index]
        val_x = train_features[val_index]
        train_target = target[train_index,ii]
        classifier = LogisticRegression(C=1000, max_iter=3000)
        classifier.fit(train_x, train_target)
        train_oof[val_index, ii] = classifier.predict_proba(val_x)[:,1]
        test_preds[:, ii] += classifier.predict_proba(test_features)[:,1]/N_FOLDS
    print(roc_auc_score(target[:,ii], train_oof[:,ii]))

Fitting target 1
Fitting fold 1
Fitting fold 2
Fitting fold 3
Fitting fold 4
Fitting fold 5
Fitting fold 6
Fitting fold 7
Fitting fold 8
Fitting fold 9
Fitting fold 10
0.9745851538638679
Fitting target 2
Fitting fold 1
Fitting fold 2
Fitting fold 3
Fitting fold 4
Fitting fold 5
Fitting fold 6
Fitting fold 7
Fitting fold 8
Fitting fold 9
Fitting fold 10
0.9827318180786321
Fitting target 3
Fitting fold 1
Fitting fold 2
Fitting fold 3
Fitting fold 4
Fitting fold 5
Fitting fold 6
Fitting fold 7
Fitting fold 8
Fitting fold 9
Fitting fold 10
0.9814986704515908
Fitting target 4
Fitting fold 1
Fitting fold 2
Fitting fold 3
Fitting fold 4
Fitting fold 5
Fitting fold 6
Fitting fold 7
Fitting fold 8
Fitting fold 9
Fitting fold 10
0.9827111268067805
Fitting target 5
Fitting fold 1
Fitting fold 2
Fitting fold 3
Fitting fold 4
Fitting fold 5
Fitting fold 6
Fitting fold 7
Fitting fold 8
Fitting fold 9
Fitting fold 10
0.9783709824221192
Fitting target 6
Fitting fold 1
Fitting fold 2
Fitting fold 3
Fit

In [6]:
test_preds

array([[9.94672471e-01, 6.61033709e-02, 9.69096386e-01, 1.03606430e-02,
        9.37980080e-01, 7.90425253e-01],
       [2.98109534e-03, 2.39571983e-04, 1.22168081e-03, 4.44214911e-07,
        9.85524140e-04, 7.79366906e-05],
       [5.04033021e-03, 2.58151400e-04, 2.12053325e-04, 9.28719813e-09,
        1.08187903e-04, 3.90319514e-05],
       ...,
       [5.52960279e-04, 3.58770616e-05, 2.22401075e-04, 3.93170539e-07,
        1.04089776e-04, 8.79660009e-05],
       [6.72047031e-03, 4.94530809e-04, 1.14434105e-02, 1.12330923e-04,
        2.26942878e-03, 2.44480029e-03],
       [8.19046497e-01, 8.67999769e-03, 3.13598293e-01, 8.83750801e-04,
        1.90008162e-01, 1.67304813e-03]])

In [7]:
submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [8]:
submission[class_names] = test_preds
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.994672,0.066103,0.969096,1.036064e-02,0.937980,0.790425
1,0000247867823ef7,0.002981,0.000240,0.001222,4.442149e-07,0.000986,0.000078
2,00013b17ad220c46,0.005040,0.000258,0.000212,9.287198e-09,0.000108,0.000039
3,00017563c3f7919a,0.000168,0.000023,0.000204,4.980453e-07,0.000411,0.000016
4,00017695ad8997eb,0.001782,0.000494,0.000201,1.212757e-03,0.000994,0.000125


In [9]:
submission.to_csv('submission.csv', index=False)